# **redis-feast-gcp**: 03 - Triton + Vertex AI Prediction Inference Example

In this notebook, we will test the deployed Triton model on the Vertex AI Prediction endpoint.

**This notebook assumes that you've already set up your Feature Store, model repo in GCP, and deployed your model in Vertex AI with NVIDIA Triton**

![architecture](img/redis-feast-gcp-architecture.png)

## Unpacking the Triton Ensemble

Before we test the inference endpoint to forecast Covid vaccinations for the state of Virginia, we will unpack the Triton Ensemble used to create the DAG of operations.

### What is an Ensemble???
An [Ensemble model](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/architecture.md#ensemble-models) represents a pipeline of one or many operations (models) and connects inputs to outputs of each stage. These are useful for inference workflows that involve several stages like data preprocessing, postprocessing, and other transformations or business logic.


![ensemble](./img/RedisFeastTriton.png)

**Checkout the structure of the Triton Model repository below.**

In [2]:
!ls ./docker/triton/models

ensemble  fetch-vaccine-features  predict-vaccine-counts


There's a model for:
- `fetch-vaccine-features` - Fetch vaccine count features from Redis at low-latency.
- `predict-vaccine-counts` - Use XGBoost model (with [Triton FIL](https://developer.nvidia.com/blog/real-time-serving-for-xgboost-scikit-learn-randomforest-lightgbm-and-more/) backend) to forecast the counts.
- `ensemble` - Wraps the other two - creating the pipeline.

Each model here has a `config.pbtxt`. Let's look at the ensemble model config below:

In [3]:
!cat ./docker/triton/models/ensemble/config.pbtxt

name: "ensemble"
platform: "ensemble"
max_batch_size: 256
input [
  {
    name: "state"
    data_type: TYPE_STRING
    dims: 1
  }
]
output [
  {
    name: "prediction"
    data_type: TYPE_FP32
    dims: 1
  }
]
ensemble_scheduling {
  step [
    {
      model_name: "fetch-vaccine-features"
      model_version: -1
      input_map {
        key: "state"
        value: "state"
      }
      output_map {
        key: "feature_values"
        value: "feature_values"
      }
    },
    {
      model_name: "predict-vaccine-counts"
      model_version: -1
      input_map {
        key: "input__0"
        value: "feature_values"
      }
      output_map {
        key: "output__0"
        value: "prediction"
      }
    }
  ]
}

## Create Inference Instances

Before we can test the Vertex AI Prediction endpoint, we need to construct a JSON body that represents an inference request. See the example below:

In [4]:
import json

# Create inference instance
payload = {
    "id": "1",
    "inputs": [
        {
            "name": "state",        ## Triton model input name
            "shape": [1, 1],        ## Triton model input shape
            "datatype": "BYTES",    ## Triton model input datatype
            "data": [["Virginia"]]  ## Triton model input data
        }
    ]
}

# Save to file
with open("instances.json", "w") as f:
    json.dump(payload, f)

## Test Endpoint

You can test the Vertex AI Prediction `rawPredict` endpoint using any HTTP tool or library, including `curl`.

In [6]:
# Log in to GCloud using the CLI and your service account
!gcloud auth activate-service-account $SERVICE_ACCOUNT_EMAIL \
    --key-file=$GOOGLE_APPLICATION_CREDENTIALS \
    --project=$PROJECT_ID

Activated service account credentials for: [tyler-redis@nifty-time-353221.iam.gserviceaccount.com]


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



In [7]:
# List out the Vertex AI endpoints
!echo $(gcloud ai endpoints list \
  --region=$GCP_REGION \
  --filter=display_name=vaccine-predictor-endpoint \
  --format="value(name)")

Using endpoint [https://us-east1-aiplatform.googleapis.com/]
4273405872979312640


In [8]:
%%bash

# Fetch Token
TOKEN=$(gcloud auth print-access-token)

# Fetch the Endpoint ID
ENDPOINT_ID=$(gcloud ai endpoints list \
  --region=$GCP_REGION \
  --filter=display_name=vaccine-predictor-endpoint \
  --format="value(name)")

# POST to the endpoint to get a response from the Triton ensemble model
curl \
  -X POST \
  -H "Authorization: Bearer ${TOKEN}" \
  -H "Content-Type: application/json" \
  https://${GCP_REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-east1/endpoints/${ENDPOINT_ID}:rawPredict \
  -d "@instances.json"


{"id":"1","model_name":"ensemble","model_version":"1","parameters":{"sequence_id":0,"sequence_start":false,"sequence_end":false},"outputs":[{"name":"prediction","datatype":"FP32","shape":[1],"data":[40990.265625]}]}

Using endpoint [https://us-east1-aiplatform.googleapis.com/]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   319    0   215  100   104    181     87  0:00:01  0:00:01 --:--:--   268


# Summary

We have just built an end-to-end ML system using Feast, Redis Enterprise, and NVIDIA Triton -- all in GCP.

This system generates realtime predictions using up to date feature values and allows us to manage "point-in-time correct" datasets from our offline datasource for training and model exploration.

Next steps that you can take after completing this tutorial include:

- Pull this repo and collaboration with your team.
- Use this tutorial to bootstrap a model for your use case by editing features / model.
- Incorporate the code in this tutorial into your company's batch pipelines by creating stages that perform feature creation and materialization.

**Redis and Triton are a perfect match: bringing the data layer (optimized for fast data access) close to the computing infrastructure (optimized for fast data processing).**